------

# <center>Data Extration & Exportation to MySQL

--------

In [ ]:
import functions as f
import os
import pandas as pd
import datetime

### Get the files

In [2]:
path='/Users/pedro/Desktop/Ironhack/Final-Project/dataverse_files/'
files=os.listdir(path)
files=[file for file in files if file.endswith('.csv')]

In [3]:
tenant_key={}
for i,file in enumerate(files):
    name,trash=file.split('.')
    name=name[3:]
    tenant_key[i]=name

invert_tenant_key={value: key for key, value in tenant_key.items()}

In [4]:
tenant_table=pd.DataFrame({'tenant':['020102d29c86','020114a6a800','020125bce03a','020149c615c5','02015d5c61cc',
                    '020177a7a91d','02018fe9be2c','02019d93db3f','0201a17a7a16','0201a8c87da4',
                    '0201b9b7d045','0201e9248493','0201f6cb55ed'],
                    'city':['Aveiro','Aveiro','Lamego','Aveiro','Aveiro','Aveiro','Lamego',
                            'Aveiro','Lamego','Aveiro','Aveiro','Aveiro','Aveiro'],
                    'start_date':['10-07-2019','02-04-2019','01-03-2019','04-03-2019','01-03-2019',
                                 '01-03-2019','03-03-2019','02-03-2019','02-04-2019','01-03-2019',
                                  '10-07-2019','01-03-2019','02-04-2019'],
                    'end_date':['02-01-2020','02-04-2020','28-02-2020','23-11-2020','13-12-2019',
                               '22-10-2020','23-11-2020','14-03-2020','02-10-2019','23-11-2020',
                               '23-11-2020','23-06-2020','23-11-2020'],
                    'rate_of_hours_with_at_least_one_log':['41%','69%','87%','62%','78%','74%',
                                                          '79%','67%','33%','81%','70%','72%','80%']})

In [5]:
tenant_table=tenant_table[tenant_table['tenant'].isin(list(tenant_key.values()))]
tenant_id=tenant_table['tenant'].map(invert_tenant_key)
tenant_table.insert(loc=0,column='tenant_id', value=tenant_id)
tenant_table.sort_values(by='tenant_id',inplace=True)

## Raw Data Extration

In [7]:
raw_data={}
for i, file in enumerate(files):
    name,trash=file.split('.')
    name=name[3:]
    data=f.pd.read_csv(path+file)
    raw_data[name]=f.extract_dfs(data)



raw_data [tenant_key [ i ] ]   [ 0 ]   **feedback**

raw_data [tenant_key [ i ] ]   [ 1 ]   **thp_sensors**

raw_data [tenant_key [ i ] ]   [ 2 ]   **dw_sensors**

raw_data [tenant_key [ i ] ]   [ 3 ]   **motion_sensors**

raw_data [tenant_key [ i ] ]   [ 4 ]   **met_cond**

i represents the tenant_id

--------

In [8]:
for i in range(0, 9):
    for j in range(0, 5):
        df = raw_data[tenant_key[i]][j]
        df['date'], df['time'] = zip(*df.index.map(f.split_index))
        df['date'] = pd.Series(df['date'], index=df.index)
        df['time'] = pd.Series(df['time'], index=df.index)
        df = f.date_time_magic(df)
        df.reset_index(drop=True, inplace=True)
        df.insert(0, 'tenant_id', i)
        
        raw_data[tenant_key[i]][j] = df

### Concatenate the DataFrames by type

In [13]:
feedback_dfs=[]
thp_sensors_dfs=[]
dw_sensors_dfs=[]
motion_sensors_dfs=[]
met_cond_dfs=[]


for i in range(9):
    df1=raw_data[tenant_key[i]][0]
    df2=raw_data[tenant_key[i]][1]
    df3=raw_data[tenant_key[i]][2]
    df4=raw_data[tenant_key[i]][3]
    df5=raw_data[tenant_key[i]][4]
    
    feedback_dfs.append(df1)
    thp_sensors_dfs.append(df2)
    dw_sensors_dfs.append(df3)
    motion_sensors_dfs.append(df4)
    met_cond_dfs.append(df5)
    
    

fb=pd.concat(feedback_dfs)
thps=pd.concat(thp_sensors_dfs)
dws=pd.concat(dw_sensors_dfs)
ms=pd.concat(motion_sensors_dfs)
mc=pd.concat(met_cond_dfs)

final_list_dfs=[fb,thps,dws,ms,mc]

for df in final_list_dfs:
    df.reset_index(drop=True, inplace=True)

In [52]:
#met_cond
met_cond=final_list_dfs[4].copy()
#thp_sensors
thp_sensors=final_list_dfs[1].copy()
#feedback
feedback=final_list_dfs[0].copy()
#dw_sensors
dw_sensors=final_list_dfs[2].copy()
#motion_sensors
motion_sensors=final_list_dfs[3].copy()

------

## <center> - END EXTRACTION -

--------

# <center>Preparing Data for MySQL

-------

## met_cond

In [55]:
met_cond[['temperature','humidity','pressure']]=met_cond[['temperature','humidity','pressure']].astype(float)

met_cond[['location','date_time']]=met_cond['description'].apply(lambda x: pd.Series(f.split_description(x)))
met_cond[['time','date']]=pd.DataFrame(met_cond['date_time'].apply(lambda x: f.date_time_met_cond(x)).to_list(), index=met_cond.index)
met_cond.drop('date_time',axis=1,inplace=True)

met_cond.drop(['windspeed','description'],axis=1,inplace=True)


In [59]:
met_cond

,tenant_id,temperature,humidity,pressure,time,date,location
0,0,9.7,84.0,1018.9,10:00:00,18-02-2019,"Porto, P.Rubras"
1,0,10.6,81.0,1019.3,11:00:00,18-02-2019,"Porto, P.Rubras"
2,0,13.3,65.0,1019.5,13:00:00,18-02-2019,"Porto, P.Rubras"
3,0,14.2,59.0,1019.1,15:00:00,18-02-2019,"Porto, P.Rubras"
4,0,14.0,56.0,1019.1,17:00:00,18-02-2019,"Porto, P.Rubras"
...,...,...,...,...,...,...,...
19838,8,15.0,76.0,NaN,22:00:00,09-10-2019,"Porto, S. Gens"
19839,8,14.8,75.0,NaN,23:00:00,09-10-2019,"Porto, S. Gens"
19840,8,18.4,91.0,NaN,11:00:00,17-10-2019,"Porto, S. Gens"
19841,8,13.6,97.0,NaN,08:00:00,18-10-2019,"Porto, S. Gens"


In [60]:
met_cond.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19843 entries, 0 to 19842
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   tenant_id    19843 non-null  int64  
 1   temperature  19843 non-null  float64
 2   humidity     19843 non-null  float64
 3   pressure     13761 non-null  float64
 4   time         19843 non-null  object 
 5   date         19843 non-null  object 
 6   location     19843 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 1.1+ MB


## thp_sensors

In [62]:
thp_sensors[['temperature', 'humidity', 'pressure']]=thp_sensors[['temperature', 'humidity', 'pressure']].astype(float)
thp_sensors.drop('linkquality',axis=1,inplace=True)
thp_sensors.drop_duplicates(inplace=True)


In [63]:
thp_sensors

,tenant_id,temperature,humidity,pressure,time,date
0,0,NaN,NaN,NaN,11:59:04,24-02-2019
2,0,NaN,NaN,NaN,11:59:09,24-02-2019
4,0,NaN,NaN,NaN,11:59:16,24-02-2019
5,0,NaN,NaN,NaN,11:59:18,24-02-2019
6,0,NaN,NaN,NaN,11:59:21,24-02-2019
...,...,...,...,...,...,...
683557,8,NaN,NaN,NaN,10:46:57,02-04-2020
683559,8,NaN,NaN,NaN,10:48:27,02-04-2020
683560,8,NaN,NaN,NaN,10:50:43,02-04-2020
683562,8,NaN,NaN,NaN,11:08:31,02-04-2020


In [64]:
thp_sensors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543879 entries, 0 to 683564
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   tenant_id    543879 non-null  int64  
 1   temperature  218785 non-null  float64
 2   humidity     218764 non-null  float64
 3   pressure     218744 non-null  float64
 4   time         543879 non-null  object 
 5   date         543879 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 29.0+ MB


## feedback

In [73]:
print('Number of rows with feedback:',feedback.shape[0]-feedback['feedback'].isna().sum())

Number of rows with feedback: 330


In [78]:
feedback.groupby('tenant_id')['device'].unique()

tenant_id
0    [02015D5C61CC, nan, status, feedback]
1              [020149C615C5, nan, status]
2              [020125BCE03A, nan, status]
3              [02019D93DB3F, nan, status]
4              [0201A17A7A16, nan, status]
5    [nan, 02018FE9BE2C, status, feedback]
6    [020177A7A91D, nan, status, feedback]
7                  [status, nan, feedback]
8              [020114A6A800, nan, status]
Name: device, dtype: object

In [79]:
feedback.dropna(subset='feedback',inplace=True)
feedback.drop(['state','device'],axis=1,inplace=True)

In [80]:
feedback

,tenant_id,feedback,time,date
135604,0,comfortable,18:49:42,12-11-2019
136544,0,comfortable,09:53:25,14-11-2019
137292,0,comfortable,12:11:46,15-11-2019
137538,0,comfortable,19:03:31,15-11-2019
140769,0,comfortable,17:04:28,23-11-2019
...,...,...,...,...
617643,7,comfortable,23:06:24,14-11-2019
617830,7,comfortable,12:25:11,15-11-2019
618135,7,comfortable,09:34:33,16-11-2019
623805,7,comfortable,13:56:15,06-12-2019


In [81]:
feedback.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 135604 to 623878
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tenant_id  330 non-null    int64 
 1   feedback   330 non-null    object
 2   time       330 non-null    object
 3   date       330 non-null    object
dtypes: int64(1), object(3)
memory usage: 12.9+ KB


## dw_sensors

In [85]:
dw_sensors.dropna(subset='linkquality',inplace=True)
dw_sensors.drop('linkquality',axis=1,inplace=True)
dw_sensors['contact'].replace({True:1,False:0},inplace=True)

In [86]:
dw_sensors

,tenant_id,contact,time,date
90,0,NaN,11:59:04,24-02-2019
91,0,NaN,11:59:04,24-02-2019
92,0,NaN,11:59:09,24-02-2019
93,0,NaN,11:59:09,24-02-2019
94,0,NaN,11:59:16,24-02-2019
...,...,...,...,...
727720,8,NaN,10:50:43,02-04-2020
727722,8,NaN,11:08:31,02-04-2020
727723,8,NaN,11:08:31,02-04-2020
727724,8,NaN,11:09:31,02-04-2020


In [87]:
dw_sensors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683566 entries, 90 to 727725
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   tenant_id  683566 non-null  int64  
 1   contact    39750 non-null   float64
 2   time       683566 non-null  object 
 3   date       683566 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 26.1+ MB


## motion_sensors

In [101]:
motion_sensors.dropna(subset='illuminance',inplace=True)

motion_sensors['occupancy'].replace({True:1,False:0},inplace=True)

motion_sensors['illuminance']=motion_sensors['illuminance'].astype(int)

In [102]:
motion_sensors

,tenant_id,illuminance,occupancy,time,date
90,0,6,NaN,11:59:04,24-02-2019
91,0,6,1.0,11:59:04,24-02-2019
92,0,1,1.0,11:59:09,24-02-2019
93,0,1,1.0,11:59:09,24-02-2019
94,0,2,1.0,11:59:16,24-02-2019
...,...,...,...,...,...
727720,8,34,1.0,10:50:43,02-04-2020
727722,8,40,1.0,11:08:31,02-04-2020
727723,8,40,1.0,11:08:31,02-04-2020
727724,8,41,1.0,11:09:31,02-04-2020


In [103]:
motion_sensors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 409250 entries, 90 to 727725
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   tenant_id    409250 non-null  int64  
 1   illuminance  409250 non-null  int64  
 2   occupancy    409153 non-null  float64
 3   time         409250 non-null  object 
 4   date         409250 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 18.7+ MB


--------

# <center>Export Data to MySQL

In [107]:
tenant_table.to_sql('tenant_table', f.engine, if_exists='replace', index=False)
met_cond.to_sql('met_cond', f.engine, if_exists='replace', index=False)
thp_sensors.to_sql('thp_sensors', f.engine, if_exists='replace', index=False)
feedback.to_sql('feedback', f.engine, if_exists='replace', index=False)
dw_sensors.to_sql('dw_sensors', f.engine, if_exists='replace', index=False)
motion_sensors.to_sql('motion_sensors', f.engine, if_exists='replace', index=False)

409250

-------

------